# metaclass
meta词根两个含义：
- 超越
- 改变

例如：YAMLObejct的一个超越变形能力，就是它的任意子类支持序列化和反序列化（serialization&deserialization）。

In [1]:
from os import name
import yaml


class Monster(yaml.YAMLObject):
    yaml_tag = u"!Monster"

    def __init__(self, name, hp, ac, attacks):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.attacks = attacks

    def __repr__(self):
        return f"{self.__class__.__name__}(name = {self.name}, hp = {self.hp}, ac = {self.ac}, attacks = {self.attacks})"

data=yaml.load("""
--- !Monster
name : Cave spider
hp : [2,6]
ac : 16
attacks : [BITE,HURT]
""",Loader=yaml.FullLoader)
print(type(data))
print(data)

print(yaml.dump(Monster(name='Cave lizard',hp=[3,6],ac=16,attacks=['BITE','HURT'])))


<class '__main__.Monster'>
Monster(name = Cave spider, hp = [2, 6], ac = 16, attacks = ['BITE', 'HURT'])
!Monster
ac: 16
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: Cave lizard



YAMLObject的特异功能体现在哪里：<br>
调用统一的yaml.load()，就能把任意一个yaml序列载入一个Python Object；而调用统一的yaml.dump()，就能把一个YAMLObject子类序列化。<br>
对于使用者来说，完全不需要提前知道任何类型信息，这让超动态配置编程编程了可能。

Python底层设计语言层面是如何实现metaclass的？<br>
- 第一，所有的python的用户定义类，都是type这个类的实例。

In [2]:
class MyClass:
    pass


instance = MyClass()

print(type(instance))

print(type(MyClass))


<class '__main__.MyClass'>
<class 'type'>


-<br>
- 第二，用户自定义类，只不过是type类的__call__运算符重载。<br>
当我们定义一个类的语句结束时，真正发生的情况，是python调用__call__运算符。简单来说，当你定义一个类时，写成下面这样：

In [ ]:
class MyClass:
    data = 1

# python 实际上执行的是这段代码
# class = type(classname,superclasses,attributedict)




这里等号右边type(classname,superclasses,attributedict)，就是type的__call__运算符重载，它会进一步调用：

In [ ]:
# type.__new__(typeclass,classname,superclasses,attributedict)
# type.__init__(class,classname,superclasses,attributedict)

验证上述：

In [9]:
class MyClass:
    data = 1


instance = MyClass()
print(MyClass)
print(instance)
print(instance.data)

MyClass = type('MyClass', (), {'data': 1})
instance = MyClass()
print(MyClass)
print(instance)
print(instance.data)


<class '__main__.MyClass'>
1
<class '__main__.MyClass'>
1


-<br>
- 第三，metaclass是type的子类，通过替换type的__call__运算符重载机制，“超越变形”正常的类。<br>
一旦你把一个类型的Myclass的metaclass设置成MyMeta，MyClass就不在由原生的type创建，而是会调用MyMeta的__call__运算符重载。

In [ ]:
# Myclass=type(classname,superclasses,attributedict)
# 变成了
# Myclass=MyMeta(classname,superclasses,attributedict)


例子

In [6]:
class MyMeta(type):
    def __init__(self, name, bases, dic):
        super().__init__(name, bases, dic)
        print("===>MyMeta.__init__")
        print(self.__name__)
        print(dic)
        print(self.yaml_tag)

    def __new__(cls, *args, **kwargs):
        print("===>MyMeta.__new__")
        print(cls.__name__)
        return type.__new__(cls, *args, **kwargs)

    # def __call__(self, *args, **kwargs):
    #     print("===>MyMeta.__call__")
    #     obj = self.__new__(self)
    #     self.__init__(self, *args, **kwargs)
    #     return obj


class Foo(metaclass=MyMeta):
    yaml_tag = "!Foo"

    def __init__(self, name):
        print("Foo.__init__")
        self.name = name

    def __new__(cls, *args, **kwargs):
        print("Foo.__new__")
        return object.__new__(cls)


foo = Foo('foo')
print(foo.name)


===>MyMeta.__new__
MyMeta
===>MyMeta.__init__
Foo
{'__module__': '__main__', '__qualname__': 'Foo', 'yaml_tag': '!Foo', '__init__': <function Foo.__init__ at 0x000001F230ADD8B0>, '__new__': <function Foo.__new__ at 0x000001F230ADD1F0>}
!Foo
Foo.__new__
Foo.__init__
foo
